# Appendix for INFO 2950 Group Project: NBA MVP Predictor
__Group Members__: Adya Bhargava (ab2446), Akhil Damani (ad674), Madeline Demers (mkd79)


**Introduction**
This file contains the collection and cleaning for the data useed in the main report. It describes and contains the code for how we collected our data by scraping the NBA site, ESPN, and another site. In order to do this we had to find data that was legal to scrape and unlocked, which proved to be a difficult process. For example, the NBA site lets users access the player data, but not the team data. Furthermore, similar site also proved to be challenging in terms of accessing the data so to get the historical list of MVPs for the past 20 years, we ended up having to scrape a text article in order to get this data. 


## Data Collection

In [132]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import duckdb
import numpy as np
import time
from unidecode import unidecode

These files were then read, cleaned, and manipulated in the phase 2 submission file. 

**Player Stats**
In this section after importing above, I collected the url and created a dataframe that collected all of the data from all of the seasons from the 2003-2004 season to the 2022-2023 season for the top player stats, ranking which players led during the season and what their stats were. I combined all of this data into one main data frame using a for loop. The data is originally filtered by average points per game for each player per season.

In [133]:
#Create a list of all of the seasons to collect data from
season_years = ["2003-04", "2004-05", "2005-06", "2006-07", "2007-08", "2008-09", "2009-10", "2010-11", \
    "2011-12", "2012-13", "2013-14", "2014-15", "2015-16", "2016-17", "2017-18", "2018-19", "2019-20", "2020-21", \
    "2021-22","2022-23"]

#Intialize empty list for dataframes to be added to
dataframes = []

#Loop through the season years and 
for season_year in season_years:

    #Add {season_year} into the api url to change each iteration
    url = f"https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=PerGame&Scope=S&Season={season_year}&SeasonType=Regular%20Season&StatCategory=PTS"
    response = requests.get(url).json()

    table_headers = response['resultSet']['headers']
    season_data = pd.DataFrame(response['resultSet']['rowSet'], columns=table_headers)

    # Do we want this to be first or last?
    season_data['Year'] = season_year

    # Append the dataframe to the list
    dataframes.append(season_data)

# Concatenate all dataframes into one giant dataframe
player_stat_df = pd.concat(dataframes, ignore_index=True)
player_stat_df


,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,OREB,DREB,REB,AST,STL,BLK,TOV,PTS,EFF,Year
0,1503,1,Tracy McGrady,1610612753,ORL,67,39.9,9.7,23.4,0.417,...,1.4,4.6,6.0,5.5,1.4,0.6,2.7,28.0,23.7,2003-04
1,978,2,Peja Stojakovic,1610612758,SAC,81,40.3,8.2,17.1,0.480,...,1.1,5.1,6.3,2.1,1.3,0.2,1.9,24.2,23.0,2003-04
2,708,3,Kevin Garnett,1610612750,MIN,82,39.4,9.8,19.6,0.499,...,3.0,10.9,13.9,5.0,1.5,2.2,2.6,24.2,33.1,2003-04
3,977,4,Kobe Bryant,1610612747,LAL,65,37.7,7.9,18.1,0.438,...,1.6,3.9,5.5,5.1,1.7,0.4,2.6,24.0,22.7,2003-04
4,1718,5,Paul Pierce,1610612738,BOS,80,38.8,7.5,18.7,0.402,...,0.9,5.7,6.5,5.1,1.6,0.7,3.8,23.0,20.5,2003-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381,200782,241,P.J. Tucker,1610612755,PHI,75,25.6,1.3,3.0,0.427,...,1.3,2.7,3.9,0.8,0.5,0.2,0.6,3.5,6.6,2022-23
4382,1630540,242,Miles McBride,1610612752,NYK,64,11.9,1.2,3.4,0.358,...,0.2,0.6,0.8,1.1,0.6,0.1,0.4,3.5,3.3,2022-23
4383,1630264,243,Anthony Gill,1610612764,WAS,59,10.6,1.2,2.2,0.538,...,0.6,1.1,1.7,0.6,0.1,0.2,0.3,3.3,4.3,2022-23
4384,1631132,244,Christian Koloko,1610612761,TOR,58,13.8,1.2,2.6,0.480,...,1.4,1.5,2.9,0.5,0.4,1.0,0.3,3.1,5.9,2022-23


**Team Stat Data Collection**
This was collected by importing csv files from the site, then we merged them in the main file. Once this data was collected and converted to a .csv file for each year, we looped through all of the years and collect the data into one main dataframe. we were able to rename columns to match the desired headers and ensure there were no string oddities, such as the '*' that had indicated a playoff team in the original site. We removed this so that it is consistent with the other data frame and then it will be easy to merge and compare the data properly. 

In [134]:
years = range(2003, 2023)

team_stats_dfs = []

for year in years:
    file_path = f'nba_team_stats/{year}.csv'
    
    team_stats_df = pd.read_csv(file_path)
    
    team_stats_df['Team'] = team_stats_df['Team'].str.replace('*', '', regex=False)

    team_stats_df['Year'] = year

    year_end = str(year + 1)
    team_stats_df['Season'] = f'{year}-{year_end[2:4]}'
    
    team_stats_dfs.append(team_stats_df)

all_team_stats_df = pd.concat(team_stats_dfs, ignore_index=True)
all_team_stats_df = all_team_stats_df[all_team_stats_df['Team'] != \
                                      'League Average']

#Accounting for any name discrepancies
all_team_stats_df.loc[all_team_stats_df['Team'] == 'Los Angeles Clippers', \
                      "Team"] = 'LA Clippers'
all_team_stats_df.loc[all_team_stats_df['Team'] == 'Charlotte Hornets', \
                      "Team"] = 'Charlotte Bobcats'

all_team_stats_df = all_team_stats_df.drop("Year", axis=1)
all_team_stats_df.rename(columns={'Season': 'SEASON'}, inplace=True)
all_team_stats_df.rename(columns={'Team': 'FULL_NAME'}, inplace=True)



all_team_stats_df


,Rk,FULL_NAME,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,AST,STL,BLK,TOV,PF,PTS,Age,W,L,SEASON
0,1.0,Dallas Mavericks,82,241.5,40.5,88.2,0.459,6.2,17.8,0.348,...,23.9,8.0,5.3,12.2,19.6,105.2,27.3,57.0,25.0,2003-04
1,2.0,Sacramento Kings,82,241.2,37.8,81.8,0.462,7.3,18.3,0.401,...,26.2,8.7,4.0,13.9,19.3,102.8,27.4,54.0,28.0,2003-04
2,3.0,Los Angeles Lakers,82,242.1,36.9,81.4,0.454,4.5,13.6,0.327,...,23.8,8.3,4.6,13.8,21.1,98.2,27.1,61.0,21.0,2003-04
3,4.0,Milwaukee Bucks,82,241.2,36.2,81.1,0.447,4.9,14.0,0.350,...,22.8,6.8,4.7,13.5,20.3,98.0,29.7,58.0,24.0,2003-04
4,5.0,Denver Nuggets,82,240.6,36.5,82.5,0.443,4.0,12.0,0.336,...,21.9,9.1,6.3,15.6,22.0,97.2,29.3,55.0,27.0,2003-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,26.0,Orlando Magic,82,241.2,40.5,86.3,0.470,10.8,31.1,0.346,...,23.2,7.4,4.7,15.1,20.1,111.4,25.1,33.0,49.0,2022-23
614,27.0,Charlotte Bobcats,82,241.8,41.3,90.4,0.457,10.7,32.5,0.330,...,25.1,7.7,5.2,14.2,20.3,111.0,25.3,27.0,55.0,2022-23
615,28.0,Houston Rockets,82,240.9,40.6,88.9,0.457,10.4,31.9,0.327,...,22.4,7.3,4.6,16.2,20.5,110.7,22.1,22.0,60.0,2022-23
616,29.0,Detroit Pistons,82,241.5,39.6,87.1,0.454,11.4,32.4,0.351,...,23.0,7.0,3.8,15.1,22.1,110.3,24.1,17.0,65.0,2022-23


**Historical MVP Data Collection**
This is the data historically ranking the MVPs over the seasons that are being analyzed. It goes through the list of target seasons and splits the text on the page to create a dataframe. This had to be done by using a text representation since the other sources were not able to be scraped. We collected data on the candidates as well as the actual MVPs. 

In [135]:
url = "https://www.nba.com/news/history-mvp-award-winners"

response = requests.get(url)

if response.status_code != 200:
        print("Something went wrong:", response.status_code, response.reason)

page = BeautifulSoup(response.content, 'html.parser')


target_seasons = ["2003-04", "2004-05", "2005-06", "2006-07", "2007-08", "2008-09", "2009-10", \
    "2010-11", "2011-12", "2012-13", "2013-14", "2014-15", "2015-16", "2016-17", "2017-18", \
    "2018-19", "2019-20", "2020-21", "2021-22", "2022-23"]

data = {} 

# Loop through the target seasons
for target_season in target_seasons:
    for p_tag in page.find_all('p'):
        if target_season in p_tag.text:
            split_data = p_tag.text.split(' — ')
            if len(split_data) > 1:
                winner = split_data[1].split(',')[0]
                data[target_season] = winner

# Create a DataFrame from the data dictionary
mvp_df = pd.DataFrame(list(data.items()), columns=["Year", "MVP_Name"])
mvp_df

,Year,MVP_Name
0,2003-04,Kevin Garnett
1,2004-05,Steve Nash
2,2005-06,Steve Nash
3,2006-07,Dirk Nowitzki
4,2007-08,Kobe Bryant
5,2008-09,LeBron James
6,2009-10,LeBron James
7,2010-11,Derrick Rose
8,2011-12,LeBron James
9,2012-13,LeBron James


In [136]:
def remove_non_numeric(input_string):
    if isinstance(input_string, int):
        return input_string
    return ''.join(char for char in input_string if char.isdigit())

In [137]:
years = range(2003, 2023)
tmp = []
for year in years:
  file_path = f'mvp_stats/{year}.csv'
    
  mvp_candidates_df = pd.read_csv(file_path, skiprows=1)
  mvp_candidates_df['Year'] = year
  year_end = str(year + 1)
  mvp_candidates_df['Season'] = f'{year}-{year_end[2:4]}'
  
  tmp.append(mvp_candidates_df)

mvp_candidates_df = pd.concat(tmp, ignore_index=True)
mvp_candidates_df['Rank'] = mvp_candidates_df['Rank'].astype(str)

mvp_candidates_df


mvp_candidates_df['is_mvp'] = 0
mvp_candidates_df['is_candidate'] = 1
mvp_candidates_df.loc[mvp_candidates_df['Rank'] == '1', 'is_mvp'] = 1
test = mvp_candidates_df[mvp_candidates_df['is_mvp'] == 1]
mvp_candidates_df.drop(columns=['Rank','Year'], inplace=True)
mvp_candidates_df.rename(columns={'Season': 'SEASON'}, inplace=True)
mvp_candidates_df.rename(columns={'Player': 'PLAYER'}, inplace=True)
mvp_candidates_df['PLAYER'] = mvp_candidates_df['PLAYER'].apply(lambda x: unidecode(str(x)) if pd.notnull(x) else x)
mvp_candidates_df.rename(columns={'Tm': 'TEAM'}, inplace=True)
mvp_candidates_df


,PLAYER,Age,TEAM,First,Pts Won,Pts Max,Share,G,MP,PTS,...,STL,BLK,FG%,3P%,FT%,WS,WS/48,SEASON,is_mvp,is_candidate
0,Kevin Garnett,27,MIN,120,1219,1230,0.991,82,39.4,24.2,...,1.5,2.2,0.499,0.256,0.791,18.3,0.272,2003-04,1,1
1,Tim Duncan,27,SAS,0,716,1230,0.582,69,36.6,22.3,...,0.9,2.7,0.501,0.167,0.599,13.1,0.249,2003-04,0,1
2,Jermaine O'Neal,25,IND,2,523,1230,0.425,78,35.7,20.1,...,0.8,2.6,0.434,0.111,0.757,9.0,0.155,2003-04,0,1
3,Peja Stojakovic,26,SAC,1,281,1230,0.228,81,40.3,24.2,...,1.3,0.2,0.480,0.433,0.927,13.5,0.198,2003-04,0,1
4,Kobe Bryant,25,LAL,0,212,1230,0.172,65,37.6,24.0,...,1.7,0.4,0.438,0.327,0.852,10.7,0.210,2003-04,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,Stephen Curry,34,GSW,0,5,1000,0.005,56,34.7,29.4,...,0.9,0.4,0.493,0.427,0.915,7.8,0.192,2022-23,0,1
271,Jimmy Butler,33,MIA,0,3,1000,0.003,64,33.4,22.9,...,1.8,0.3,0.539,0.350,0.850,12.3,0.277,2022-23,0,1
272,De'Aaron Fox,25,SAC,0,2,1000,0.002,73,33.4,25.0,...,1.1,0.3,0.512,0.324,0.780,7.4,0.146,2022-23,0,1
273,Jalen Brunson,26,NYK,0,1,1000,0.001,68,35.0,24.0,...,0.9,0.2,0.491,0.416,0.829,8.7,0.175,2022-23,0,1


## Data Cleaning


**Player Stat Data Cleaning** 
In the 'data_collection.ipynb' file we were able to collect this data from the NBA stats site and then clean it up so as it met our needs. By reading the .csv file we had a large set of data that was of very good quality, documenting the past 20 seasons statistics for all of the players in the league during the regular season. Below, we dropped some unecessary columns that prove irrelevant to our analysis as well as rename some of the others to be more consistent with the headers across the board. A preview of the data frame is shown below.

In [138]:
#player_stats_df = pd.read_csv('player_stats.csv')

player_stat_df = player_stat_df.drop("PLAYER_ID", axis=1)
player_stat_df = player_stat_df.drop("TEAM_ID", axis=1)
player_stat_df = player_stat_df.drop("RANK", axis=1)

player_stat_df.rename(columns={'Year': 'SEASON'}, inplace=True)

player_stat_df

,PLAYER,TEAM,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,...,OREB,DREB,REB,AST,STL,BLK,TOV,PTS,EFF,SEASON
0,Tracy McGrady,ORL,67,39.9,9.7,23.4,0.417,2.6,7.7,0.339,...,1.4,4.6,6.0,5.5,1.4,0.6,2.7,28.0,23.7,2003-04
1,Peja Stojakovic,SAC,81,40.3,8.2,17.1,0.480,3.0,6.8,0.433,...,1.1,5.1,6.3,2.1,1.3,0.2,1.9,24.2,23.0,2003-04
2,Kevin Garnett,MIN,82,39.4,9.8,19.6,0.499,0.1,0.5,0.256,...,3.0,10.9,13.9,5.0,1.5,2.2,2.6,24.2,33.1,2003-04
3,Kobe Bryant,LAL,65,37.7,7.9,18.1,0.438,1.1,3.3,0.327,...,1.6,3.9,5.5,5.1,1.7,0.4,2.6,24.0,22.7,2003-04
4,Paul Pierce,BOS,80,38.8,7.5,18.7,0.402,1.4,4.8,0.299,...,0.9,5.7,6.5,5.1,1.6,0.7,3.8,23.0,20.5,2003-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381,P.J. Tucker,PHI,75,25.6,1.3,3.0,0.427,0.7,1.9,0.393,...,1.3,2.7,3.9,0.8,0.5,0.2,0.6,3.5,6.6,2022-23
4382,Miles McBride,NYK,64,11.9,1.2,3.4,0.358,0.6,2.1,0.299,...,0.2,0.6,0.8,1.1,0.6,0.1,0.4,3.5,3.3,2022-23
4383,Anthony Gill,WAS,59,10.6,1.2,2.2,0.538,0.1,0.5,0.138,...,0.6,1.1,1.7,0.6,0.1,0.2,0.3,3.3,4.3,2022-23
4384,Christian Koloko,TOR,58,13.8,1.2,2.6,0.480,0.0,0.2,0.083,...,1.4,1.5,2.9,0.5,0.4,1.0,0.3,3.1,5.9,2022-23


In order to improve the process of merging, sorting, and analyzing our data we wanted to be sure that the abbreviations presented in the original file did not impede on our ability to merge with other dataframes we have such as the team statistics. So, we added a column mapping the full team name for each abbreviation. This also considered teams that have changed locations/names over the 20 year span in order to be inclusive of all relevant names and abbreviations in the data set. The column was also renamed and moved closer to the one it shares similarities with as for visual ease of use.

In [139]:
team_name_mapping = {
    'ORL': 'Orlando Magic',
    'SAC': 'Sacramento Kings',
    'MIN': 'Minnesota Timberwolves',
    'LAL': 'Los Angeles Lakers',
    'BOS': 'Boston Celtics',
    'NOH': 'New Orleans Hornets',
    'TOR': 'Toronto Raptors',
    'SAS': 'San Antonio Spurs',
    'DAL': 'Dallas Mavericks',
    'MIL': 'Milwaukee Bucks',
    'DEN': 'Denver Nuggets',
    'CLE': 'Cleveland Cavaliers',
    'LAC': 'LA Clippers',
    'NYK': 'New York Knicks',
    'IND': 'Indiana Pacers',
    'POR': 'Portland Trail Blazers',
    'PHX': 'Phoenix Suns',
    'GSW': 'Golden State Warriors',
    'NJN': 'New Jersey Nets',
    'ATL': 'Atlanta Hawks',
    'SEA': 'Seattle SuperSonics',
    'MEM': 'Memphis Grizzlies',
    'DET': 'Detroit Pistons',
    'HOU': 'Houston Rockets',
    'MIA': 'Miami Heat',
    'CHI': 'Chicago Bulls',
    'UTA': 'Utah Jazz',
    'PHI': 'Philadelphia 76ers',
    'WAS': 'Washington Wizards',
    'CHA': 'Charlotte Bobcats',
    'NOK': 'New Orleans/Oklahoma City Hornets',
    'OKC': 'Oklahoma City Thunder',
    'BKN': 'Brooklyn Nets',
    'NOP': 'New Orleans Pelicans'
}

player_stat_df['FULL_NAME'] = player_stat_df['TEAM'].map(team_name_mapping)
full_name_column = player_stat_df.pop("FULL_NAME")
player_stat_df.insert(player_stat_df.columns.get_loc("TEAM") + 1,\
                        "FULL_NAME", full_name_column)

player_stat_df

,PLAYER,TEAM,FULL_NAME,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,...,OREB,DREB,REB,AST,STL,BLK,TOV,PTS,EFF,SEASON
0,Tracy McGrady,ORL,Orlando Magic,67,39.9,9.7,23.4,0.417,2.6,7.7,...,1.4,4.6,6.0,5.5,1.4,0.6,2.7,28.0,23.7,2003-04
1,Peja Stojakovic,SAC,Sacramento Kings,81,40.3,8.2,17.1,0.480,3.0,6.8,...,1.1,5.1,6.3,2.1,1.3,0.2,1.9,24.2,23.0,2003-04
2,Kevin Garnett,MIN,Minnesota Timberwolves,82,39.4,9.8,19.6,0.499,0.1,0.5,...,3.0,10.9,13.9,5.0,1.5,2.2,2.6,24.2,33.1,2003-04
3,Kobe Bryant,LAL,Los Angeles Lakers,65,37.7,7.9,18.1,0.438,1.1,3.3,...,1.6,3.9,5.5,5.1,1.7,0.4,2.6,24.0,22.7,2003-04
4,Paul Pierce,BOS,Boston Celtics,80,38.8,7.5,18.7,0.402,1.4,4.8,...,0.9,5.7,6.5,5.1,1.6,0.7,3.8,23.0,20.5,2003-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381,P.J. Tucker,PHI,Philadelphia 76ers,75,25.6,1.3,3.0,0.427,0.7,1.9,...,1.3,2.7,3.9,0.8,0.5,0.2,0.6,3.5,6.6,2022-23
4382,Miles McBride,NYK,New York Knicks,64,11.9,1.2,3.4,0.358,0.6,2.1,...,0.2,0.6,0.8,1.1,0.6,0.1,0.4,3.5,3.3,2022-23
4383,Anthony Gill,WAS,Washington Wizards,59,10.6,1.2,2.2,0.538,0.1,0.5,...,0.6,1.1,1.7,0.6,0.1,0.2,0.3,3.3,4.3,2022-23
4384,Christian Koloko,TOR,Toronto Raptors,58,13.8,1.2,2.6,0.480,0.0,0.2,...,1.4,1.5,2.9,0.5,0.4,1.0,0.3,3.1,5.9,2022-23


The "SEASON" column was moved to provide an easier visual glance at the data upon looking at the large data frame in a form such as the one shown below. Then, this column was changed to a datetime object to ensure that in the future this will help us have well organized and formatted data.

In [140]:
season_column = player_stat_df.pop("SEASON")
player_stat_df.insert(player_stat_df.columns.get_loc("FULL_NAME") + 1, \
                       "SEASON", season_column)
player_stat_df['PLAYER'] = player_stat_df['PLAYER'].apply(lambda x: unidecode(str(x)) if pd.notnull(x) else x)

player_stat_df

,PLAYER,TEAM,FULL_NAME,SEASON,GP,MIN,FGM,FGA,FG_PCT,FG3M,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PTS,EFF
0,Tracy McGrady,ORL,Orlando Magic,2003-04,67,39.9,9.7,23.4,0.417,2.6,...,0.796,1.4,4.6,6.0,5.5,1.4,0.6,2.7,28.0,23.7
1,Peja Stojakovic,SAC,Sacramento Kings,2003-04,81,40.3,8.2,17.1,0.480,3.0,...,0.927,1.1,5.1,6.3,2.1,1.3,0.2,1.9,24.2,23.0
2,Kevin Garnett,MIN,Minnesota Timberwolves,2003-04,82,39.4,9.8,19.6,0.499,0.1,...,0.791,3.0,10.9,13.9,5.0,1.5,2.2,2.6,24.2,33.1
3,Kobe Bryant,LAL,Los Angeles Lakers,2003-04,65,37.7,7.9,18.1,0.438,1.1,...,0.852,1.6,3.9,5.5,5.1,1.7,0.4,2.6,24.0,22.7
4,Paul Pierce,BOS,Boston Celtics,2003-04,80,38.8,7.5,18.7,0.402,1.4,...,0.819,0.9,5.7,6.5,5.1,1.6,0.7,3.8,23.0,20.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381,P.J. Tucker,PHI,Philadelphia 76ers,2022-23,75,25.6,1.3,3.0,0.427,0.7,...,0.826,1.3,2.7,3.9,0.8,0.5,0.2,0.6,3.5,6.6
4382,Miles McBride,NYK,New York Knicks,2022-23,64,11.9,1.2,3.4,0.358,0.6,...,0.667,0.2,0.6,0.8,1.1,0.6,0.1,0.4,3.5,3.3
4383,Anthony Gill,WAS,Washington Wizards,2022-23,59,10.6,1.2,2.2,0.538,0.1,...,0.731,0.6,1.1,1.7,0.6,0.1,0.2,0.3,3.3,4.3
4384,Christian Koloko,TOR,Toronto Raptors,2022-23,58,13.8,1.2,2.6,0.480,0.0,...,0.627,1.4,1.5,2.9,0.5,0.4,1.0,0.3,3.1,5.9


In [141]:

player_stats_df = pd.merge(player_stat_df, mvp_candidates_df[['PLAYER', 'SEASON', 'is_mvp', 'is_candidate']],
                     on=['PLAYER', 'SEASON'], how='left')
player_stats_df['is_mvp'].fillna(0, inplace=True)
player_stats_df['is_candidate'].fillna(0, inplace=True)

player_stats_df

,PLAYER,TEAM,FULL_NAME,SEASON,GP,MIN,FGM,FGA,FG_PCT,FG3M,...,DREB,REB,AST,STL,BLK,TOV,PTS,EFF,is_mvp,is_candidate
0,Tracy McGrady,ORL,Orlando Magic,2003-04,67,39.9,9.7,23.4,0.417,2.6,...,4.6,6.0,5.5,1.4,0.6,2.7,28.0,23.7,0.0,0.0
1,Peja Stojakovic,SAC,Sacramento Kings,2003-04,81,40.3,8.2,17.1,0.480,3.0,...,5.1,6.3,2.1,1.3,0.2,1.9,24.2,23.0,0.0,1.0
2,Kevin Garnett,MIN,Minnesota Timberwolves,2003-04,82,39.4,9.8,19.6,0.499,0.1,...,10.9,13.9,5.0,1.5,2.2,2.6,24.2,33.1,1.0,1.0
3,Kobe Bryant,LAL,Los Angeles Lakers,2003-04,65,37.7,7.9,18.1,0.438,1.1,...,3.9,5.5,5.1,1.7,0.4,2.6,24.0,22.7,0.0,1.0
4,Paul Pierce,BOS,Boston Celtics,2003-04,80,38.8,7.5,18.7,0.402,1.4,...,5.7,6.5,5.1,1.6,0.7,3.8,23.0,20.5,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381,P.J. Tucker,PHI,Philadelphia 76ers,2022-23,75,25.6,1.3,3.0,0.427,0.7,...,2.7,3.9,0.8,0.5,0.2,0.6,3.5,6.6,0.0,0.0
4382,Miles McBride,NYK,New York Knicks,2022-23,64,11.9,1.2,3.4,0.358,0.6,...,0.6,0.8,1.1,0.6,0.1,0.4,3.5,3.3,0.0,0.0
4383,Anthony Gill,WAS,Washington Wizards,2022-23,59,10.6,1.2,2.2,0.538,0.1,...,1.1,1.7,0.6,0.1,0.2,0.3,3.3,4.3,0.0,0.0
4384,Christian Koloko,TOR,Toronto Raptors,2022-23,58,13.8,1.2,2.6,0.480,0.0,...,1.5,2.9,0.5,0.4,1.0,0.3,3.1,5.9,0.0,0.0


Merge Player Stats df with Team Stats to get W, L 

In [142]:

player_stats_df = pd.merge(player_stats_df, all_team_stats_df[['FULL_NAME','SEASON','W', 'L']],
                     on=['FULL_NAME','SEASON'], how='left')

player_stats_df


,PLAYER,TEAM,FULL_NAME,SEASON,GP,MIN,FGM,FGA,FG_PCT,FG3M,...,AST,STL,BLK,TOV,PTS,EFF,is_mvp,is_candidate,W,L
0,Tracy McGrady,ORL,Orlando Magic,2003-04,67,39.9,9.7,23.4,0.417,2.6,...,5.5,1.4,0.6,2.7,28.0,23.7,0.0,0.0,41.0,41.0
1,Peja Stojakovic,SAC,Sacramento Kings,2003-04,81,40.3,8.2,17.1,0.480,3.0,...,2.1,1.3,0.2,1.9,24.2,23.0,0.0,1.0,54.0,28.0
2,Kevin Garnett,MIN,Minnesota Timberwolves,2003-04,82,39.4,9.8,19.6,0.499,0.1,...,5.0,1.5,2.2,2.6,24.2,33.1,1.0,1.0,45.0,37.0
3,Kobe Bryant,LAL,Los Angeles Lakers,2003-04,65,37.7,7.9,18.1,0.438,1.1,...,5.1,1.7,0.4,2.6,24.0,22.7,0.0,1.0,61.0,21.0
4,Paul Pierce,BOS,Boston Celtics,2003-04,80,38.8,7.5,18.7,0.402,1.4,...,5.1,1.6,0.7,3.8,23.0,20.5,0.0,0.0,47.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381,P.J. Tucker,PHI,Philadelphia 76ers,2022-23,75,25.6,1.3,3.0,0.427,0.7,...,0.8,0.5,0.2,0.6,3.5,6.6,0.0,0.0,40.0,42.0
4382,Miles McBride,NYK,New York Knicks,2022-23,64,11.9,1.2,3.4,0.358,0.6,...,1.1,0.6,0.1,0.4,3.5,3.3,0.0,0.0,44.0,38.0
4383,Anthony Gill,WAS,Washington Wizards,2022-23,59,10.6,1.2,2.2,0.538,0.1,...,0.6,0.1,0.2,0.3,3.3,4.3,0.0,0.0,44.0,38.0
4384,Christian Koloko,TOR,Toronto Raptors,2022-23,58,13.8,1.2,2.6,0.480,0.0,...,0.5,0.4,1.0,0.3,3.1,5.9,0.0,0.0,34.0,48.0


REPLACE THIS WITH THE UPDATED DATA FRAME (player_stats_df)
**Merged players and team data**
Merged the dataframes of the player stats along with the team stats




In [143]:
#all_team_stats_df = pd.read_csv('all_team_stats.csv')
# merged_df = duckdb.sql("SELECT * FROM player_stat_df p LEFT JOIN \
# all_team_stats_df t ON p.FULL_NAME = t.TEAM AND p.SEASON = t.SEASON").df()

# merged_df[merged_df["SEASON"]=="2021-22"]
# merged_df[merged_df["PLAYER"]=="Nikola Jokic"]

Merged the mvp team data with the team stats

In [144]:
# mvp_teams_df = duckdb.sql("SELECT * FROM mvp_team_df m LEFT JOIN \
#     all_team_stats_df a ON m.FULL_NAME = a.TEAM AND m.SEASON = a.SEASON").df()

# mvp_teams_df

**Model Data**

In [145]:
# model_df = merged_df
# model_df = duckdb.sql("SELECT x.*, y.MVP, CASE WHEN y.Rank IS NOT NULL THEN 1 ELSE 0\
#  END AS MVP_CANDIDATE, COALESCE(y.Share, 0) AS VOTE_SHARE FROM model_df x LEFT JOIN mvp_stats_df y \
#                   ON x.PLAYER = y.Player AND x.SEASON = y.Season").df()

# model_df["MVP"] = model_df["MVP"].fillna(0)
# model_df


# Conversion to .csv File


In [146]:
mvp_df.to_csv('mvp_df.csv')
mvp_candidates_df.to_csv('mvp_candidates.csv')
player_stats_df.to_csv('player_stats.csv')
all_team_stats_df.to_csv('all_team_stats.csv')


#model_df.to_csv('model.csv')

